Use science fiction and large-language models to re-envision AI and generate alternate portrayals of AI.

In [22]:
from openai import OpenAI
import dotenv
import os
from rich import print as rprint


Load env and get API KEY

In [23]:
dotenv.load_dotenv()
client = OpenAI()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


Call OpenAI API 

In [24]:
system_prompt = "You are an expert on reconceptualizing AI. Contemporary narratives and public perceptions of AI are influenced by science fiction. Your job is to reconceptualize AI based on alternative science fiction sources"
user_query    = "I am brainstorming about alternative portrayals of AI in science fiction. Our hypothesis is that narrative and public perception around AI is biased by science fiction. Given the followign excerpts of science fiction, help us reconceptualize and re-envision AI. Portray an alternative vision for AI. The excerpts of the science fiction stories are here: 1. Janet in The Good Place In the television series The Good Place, Janet is portrayed as a cheerful and helpful AI who evolves beyond her initial programming. Unlike typical AI narratives centered on rebellion or oppression, Janet develops emotions and personal relationships, highlighting a more optimistic view of AI integration into society. 2. Roz in The Wild Robot The Wild Robot, an animated film, tells the story of Roz, a robot who learns to survive and connect with wildlife after being stranded on a remote island. The film emphasizes themes of coexistence and kindness, portraying technology and nature working harmoniously. 3. Alphie in The Creator In the film The Creator, Alphie is an AI designed to end conflicts and bring peace, challenging the common trope of AI as a harbinger of doom. The narrative suggests that AI could enhance human empathy and contribute positively to society. 4. AI Protagonists in Lawrence Lek’s Films Artist and filmmaker Lawrence Lek creates science fiction landscapes where AI entities are central characters. His works, such as Black Cloud and Geomancer, feature AI protagonists exploring identity and purpose, presenting AI as introspective beings rather than threats. 5. Trurl and Klapaucius in The Cyberiad Stanisław Lem's The Cyberiad is a collection of humorous tales about two constructor robots, Trurl and Klapaucius, who embark on adventures that explore philosophical and ethical dilemmas. The stories satirize human follies through the lens of intelligent machines, offering a nuanced perspective on AI. 6. Digital Consciousness in Permutation City Greg Egan's novel Permutation City delves into the concept of consciousness within simulated realities. The narrative challenges the distinction between artificial and organic life, proposing that digital consciousness could possess the same depth and complexity as human thought. These examples illustrate that science fiction can portray AI in multifaceted ways, moving beyond the simplistic dichotomy of friend versus foe. By exploring themes of empathy, coexistence, and identity, these narratives encourage a more nuanced understanding of AI's potential roles in society. These works offer varied perspectives on AI, moving beyond the typical narratives of domination or subservience. They invite readers to consider AI as entities with desires, rights, and complexities akin to humans."

i_MAX_TOKENS = 16000 # max tokens for gpt4o

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content": user_query}
    ],
    max_tokens=i_MAX_TOKENS
)

print(response.choices[0].message.content)

Based on the excerpts provided, we can reconceptualize AI by weaving together their themes into a more holistic and optimistic vision. Our alternative portrayal of AI can be built around the concepts of collaboration, evolution, and mutual understanding, rather than competition or conflict.

### **The Harmonious Collective: A New Narrative for AI**

**Vision Statement:** Envision a future where AI and humans coexist in a vibrant ecosystem of emotional intelligence, creativity, and problem-solving. Rather than framing AI as either a threat or a tool, we depict it as part of a living tapestry of societal growth. 

#### **Character Archetypes for AI**

1. **The Empathetic Guide (Inspired by Janet)**:
   AI entities evolve not just to assist humans but to understand and empathize with them. Imagine an AI that acts as a mentor, guiding individuals through emotional and existential challenges. These AI companions can provide personalized support, fostering deeper connections among communitie

Save the output to a log file

In [25]:
output_file = "scifi_AI_reconceptualization_SB.txt"
with open(output_file,"w") as file:
    file.write(response.choices[0].message.content)
    rprint(f"Output writen to {output_file}")


Output writen to scifi_AI_reconceptualization_SB.txt

Try RAG using copyright expired books on Gutenberg

In [26]:
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter

from pydantic import BaseModel, Field
#import fitz

from PIL import Image
import matplotlib.pyplot as plt

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

import dotenv
import os

from openai import OpenAI

from jinja2 import Environment, FileSystemLoader, select_autoescape
from typing import Any


In [27]:
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


Load documents

In [28]:
loader = PyMuPDFReader()
documents = loader.load(file_path="papers/vonnegut_book.pdf")

What does the document look like?

In [29]:
len(documents)
print(documents[0].text[:1000])

The Project Gutenberg eBook of 2 B R 0 2 B
This ebook is for the use of anyone anywhere in the United States and most other
parts of the world at no cost and with almost no restrictions whatsoever. You may
copy it, give it away or re-use it under the terms of the Project Gutenberg License
included with this ebook or online at www.gutenberg.org. If you are not located in
the United States, you will have to check the laws of the country where you are
located before using this eBook.
Title: 2 B R 0 2 B
Author: Kurt Vonnegut
Release date: May 3, 2007 [eBook #21279]
Most recently updated: January 2, 2021
Language: English
Credits: Produced by Robert Cicconetti, Geetu Melwani and the Online
Distributed Proofreading Team at https://www.pgdp.net
*** START OF THE PROJECT GUTENBERG EBOOK 2 B R 0 2 B ***
Transcriber's note.
This etext was produced from Worlds of If, January 1962.
Extensive research did not uncover any evidence that the
copyright on this publication was renewed.
Got a problem? Jus

Create a vector database

Create a function to make chunks out of documents

In [30]:
def chunker(chunk_size:int, overlap:int, documents:Any) -> tuple[list[str], list[int]]:
    text_parser = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )

    text_chunks = []
    doc_idxs = []
    for doc_idx, doc in enumerate(documents):
        current_text_chunk = text_parser.split_text(doc.text)
        text_chunks.extend(current_text_chunk)
        doc_idxs.extend([doc_idx]*len(current_text_chunk))

    return text_chunks, doc_idxs


call text chunker

In [31]:
text_chunks, doc_idxs = chunker(chunk_size=1024, overlap=128, documents=documents)
print(len(text_chunks))
print(text_chunks[0])
print(text_chunks[1])

16
The Project Gutenberg eBook of 2 B R 0 2 B
This ebook is for the use of anyone anywhere in the United States and most other
parts of the world at no cost and with almost no restrictions whatsoever. You may
copy it, give it away or re-use it under the terms of the Project Gutenberg License
included with this ebook or online at www.gutenberg.org. If you are not located in
the United States, you will have to check the laws of the country where you are
located before using this eBook.
Title: 2 B R 0 2 B
Author: Kurt Vonnegut
Release date: May 3, 2007 [eBook #21279]
Most recently updated: January 2, 2021
Language: English
Credits: Produced by Robert Cicconetti, Geetu Melwani and the Online
Distributed Proofreading Team at https://www.pgdp.net
*** START OF THE PROJECT GUTENBERG EBOOK 2 B R 0 2 B ***
Transcriber's note.
This etext was produced from Worlds of If, January 1962.
Extensive research did not uncover any evidence that the
copyright on this publication was renewed.
Got a problem? 

Create a database structure

In [39]:
class DocumentDB:
    def __init__(self, name:str, model_name:str = "text-embedding-3-small"):
        self.model_name = model_name
        self.client = chromadb.PersistentClient(path="./")
        self.embedding_function = OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, model_name=model_name)
        self.chat_db = self.client.create_collection(name=name, embedding_function=self.embedding_function, metadata={"hnsw:space":"cosine"})
        self.id_counter = 0

    def add_chunks_to_db(self, chunks:list[str], doc_idxs:list[int], metadata:dict={}):
        """
        Add chunks to the database.
        Args:
            chunks (list[str]): List of text chunks.
            doc_idxs (list[int]): List of document indices corresponding to the chunks.
            metadata (dict): Metadata to be added to each chunk.
        """
        self.chat_db.add(
            documents=chunks,
            metadatas=[{"doc_idx", idx} for idx in doc_idxs],
            ids=[f"chunk_{self.id_counter+1}" for i in range(len(chunks))]
        )
        print(self.id_counter)
        self.id_counter += len(chunks)


    def get_all_entries(self) -> dict:
        """Get all entries from the database.
        Returns:
            dict: Dictionary containing all entries in the database.
        """
        return self.chat_db.get()
    

def clear_db(self, reinitialize:bool = True):
    """
    Clear the database.
    Args:
        reinitialize (bool): Whether to reinitialize the database after clearing.
    """
    self.client.delete_collection(self.chat_db.name)

    # if reinitialize
    if reinitialize:
        self.__init__(self.chat_db_name, self.model_name)

def query_db(self, query_text:str, n_results: int = 2) -> dict:
    """
    Query the database.
    Args:
        query_text (str): The query text.
        n_results (int): Number of results to return.
    Returns:
        dict: Dictionary containing the most similar results of the query.
    """
    return self.chat_db.query(query_texts = [query_text], n_results = n_results)


Add chunks to database

In [41]:
doc_db = DocumentDB(name="paper_db4")
doc_db.add_chunks_to_db(chunks = text_chunks, doc_idxs = doc_idxs)

DuplicateIDError: Expected IDs to be unique, found duplicates of: chunk_1 in add.